# Code Part for plotting the respective Graphs. 
The aim is to plot the **Height-per-Age** and **Weight-per-Age** profiles for different subgroups. 
For this, we use the statsmodels nonparametric lowess function and build confidence intervals via bootstrap, as the required function (STATA: twoway polyfit) is not available in python. 

Importing the required libraries    

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.utils import resample
from scipy.interpolate import interp1d  # For smooth interpolation
from pathlib import Path  # For file path management


Load the data 

In [ ]:
data = pd.read_csv(data_path)

Function for local polynomial regression: 

In [ ]:
def local_poly_regression(x, y, degree=3, frac=0.3, x_eval=None):
    """
    Perform local polynomial regression (lowess).

    Parameters:
    -----------
    x, y : array-like
        Input data.
    degree : int
        Degree of polynomial (not used directly here since lowess is nonparametric).
    frac : float
        Fraction of points used for smoothing.
    x_eval : array-like
        Points to evaluate the fitted curve.

    Returns:
    --------
    x_eval : array-like
        Evaluated x-points.
    fit_y : array-like
        Smoothed y-values.
    """
    if x_eval is None:
        x_eval = np.linspace(min(x), max(x), 500)  # High-resolution x-values
    fit = sm.nonparametric.lowess(y, x, frac=frac, return_sorted=True)
    fit_interp = interp1d(fit[:, 0], fit[:, 1], fill_value="extrapolate")
    return x_eval, fit_interp(x_eval)

Function to compute bootstrap confidence intervals:

In [ ]:
def bootstrap_ci(x, y, degree=3, frac=0.3, n_bootstrap=1000, ci=95, x_eval=None):
    """
    Compute bootstrap confidence intervals for local polynomial regression.

    Parameters:
    -----------
    x, y : array-like
        Input data.
    degree : int
        Degree of polynomial (not used directly here since lowess is nonparametric).
    frac : float
        Fraction of points used for smoothing.
    n_bootstrap : int
        Number of bootstrap iterations.
    ci : int
        Confidence level (e.g., 95 for 95% CI).
    x_eval : array-like
        Points to evaluate the fitted curve.

    Returns:
    --------
    lower_bound : array-like
        Lower bound of confidence interval.
    upper_bound : array-like
        Upper bound of confidence interval.
    """
    if x_eval is None:
        x_eval = np.linspace(min(x), max(x), 500)  # High-resolution x-values

    boot_coefs = np.zeros((n_bootstrap, len(x_eval)))
    for i in range(n_bootstrap):
        x_resampled, y_resampled = resample(x, y)
        _, boot_fit = local_poly_regression(x_resampled, y_resampled, degree=degree, frac=frac, x_eval=x_eval)
        boot_coefs[i, :] = boot_fit

    # Calculate lower and upper percentiles for the confidence intervals
    lower_bound = np.percentile(boot_coefs, (100 - ci) / 2, axis=0)
    upper_bound = np.percentile(boot_coefs, 100 - (100 - ci) / 2, axis=0)
    return lower_bound, upper_bound

Function to plot WHO Z-scores with confidence intervals

In [ ]:
def plot_zscores(data, subgroup_column, cases, x_col, degree=3, frac=0.3, ci=95, n_bootstrap=1000):
    """
    Plot WHO Z-scores with confidence intervals for specified cases and subgroups.

    Parameters:
    -----------
    data : pd.DataFrame
        Input data.
    subgroup_column : str
        Column name for the subgroup (e.g., 'urban', 'ar', 'male').
    cases : dict
        Dictionary of case names to column names (e.g., {'HAZ': 'hta', 'WAZ': 'wta'}).
    x_col : str
        Column name for the x-axis variable (e.g., 'childage').
    degree : int
        Degree of the polynomial for smoothing.
    frac : float
        Fraction of points used for lowess regression.
    ci : int
        Confidence interval percentage.
    n_bootstrap : int
        Number of bootstrap iterations.
    """
    plt.figure(figsize=(12, 8))
    for subgroup, subgroup_data in data.groupby(subgroup_column):
        x_eval = np.linspace(subgroup_data[x_col].min(), subgroup_data[x_col].max(), 500)
        for case_name, case_column in cases.items():
            x = subgroup_data[x_col]
            y = subgroup_data[case_column]
            
            # Perform local polynomial regression
            fit_x, fit_y = local_poly_regression(x, y, degree=degree, frac=frac, x_eval=x_eval)
            
            # Compute confidence intervals
            lower_ci, upper_ci = bootstrap_ci(x, y, degree=degree, frac=frac, x_eval=x_eval, n_bootstrap=n_bootstrap, ci=ci)
            
            # Plot
            plt.plot(fit_x, fit_y, label=f'{case_name} ({subgroup})')
            plt.fill_between(fit_x, lower_ci, upper_ci, alpha=0.3, label=f'95% CI {case_name} ({subgroup})')

    plt.title(f'WHO Z-score by Age of Child ({subgroup_column})')
    plt.xlabel('Age of the child in months')
    plt.ylabel('WHO Z-score')
    plt.legend()
    plt.grid(True, linestyle='--', color='lightgray')
    plt.savefig(f'ageprofiles_{subgroup_column}_cases.pdf')
    plt.show()

Define cases and columns for analysis

In [ ]:
cases = {'HAZ': 'hta', 'WAZ': 'wta'}

Plot for different subgroups

In [ ]:

plot_zscores(data, subgroup_column='ar', cases=cases, x_col='childage')  # Affected region
plot_zscores(data, subgroup_column='male', cases=cases, x_col='childage')  # Gender
plot_zscores(data, subgroup_column='urban', cases=cases, x_col='childage')  # Urban vs rural
